In [1]:
import pandas as pd
import keyring
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap
from bokeh.layouts import column
from bokeh.io import show

In [ ]:
df = pd.read_csv('/Users/Felis.catus/Desktop/CAT001_D03_P02.csv')
df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
df['date'] = df['timestamp'].dt.date
df['_date'] = df['date']
df['date'] = df['date'].astype('datetime64[ns]')
dates = df['date'].unique()
df = df.set_index('date')
df = df.loc[(df['gps:hdop'] < 1.3)]

In [ ]:
def plot(start, end):
    df_ = df.loc[start:end]
    start = str(pd.to_datetime(df_['_date'][0]).date())
    end = str(pd.to_datetime(df_['_date'][-1]).date())
    map_options = GMapOptions(
        lat=list(df_['location-lat'])[0],
        lng=list(df_['location-long'])[0],
        map_type="hybrid",
        zoom=18
    )
    api_key =  keyring.get_password('google_maps', 'API_KEY')
    p = gmap(api_key, map_options, title=f'{start} to {end}')
    source = ColumnDataSource(
        data=dict(lat=list(df_['location-lat']),
                  lon=df_['location-long'])
    )
    p.circle(
        x='lon',
        y='lat',
        size=5,
        fill_color='red',
        line_color='red',
        fill_alpha=1,
        source=source
    )
    return p

objs = [plot(dates[n], dates[n + 1]) for n in range(0, len(dates), 2)]
show(column(*objs))